In [1]:
from Levenshtein import distance as levenshtein_distance
import time
import torch
import numpy as np
from torch.optim import Adam
from options import *
from Encoder import *
from Decoder import *
from util import *


In [29]:
options = BaseOptions()
opt = options.parser.parse_args(args = [])

speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")

word_dict, word_list, transcript_train, transcript_valid = collect_word(transcript_train, transcript_valid)
opt.vocab_size = len(word_list)
options.printer(opt)
print("Transfer the transcript from words to index sucessfully.....")

encoder = Encoder(opt)
decoder = Decoder(opt)
# encoder.load_state_dict(torch.load('./pre_train_model/encoder_pretrained.pt'))
# decoder.load_state_dict(torch.load('./pre_train_model/decoder_pretrained.pt'))
encoder.to(opt.device)
decoder.to(opt.device)
print(decoder)
optimizer_encoder = Adam(encoder.parameters(), opt.lr, weight_decay = 1e-6)
optimizer_decoder = Adam(decoder.parameters(), opt.lr, weight_decay = 1e-6)
criterion = nn.CrossEntropyLoss(reduction = 'none')
criterion.to(opt.device)


train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

# training(opt, encoder, decoder, train_loader, dev_loader)


Data Loading Sucessful.....
----------------- Options ---------------
                 dataroot: ./data/                       
       decoder_hidden_dim: 512                           
       decoder_num_layers: 2                             
                   device: cuda                          
             display_freq: 10                            
                  dropout: 0.5                           
           embedding_size: 256                           
       encoder_hidden_dim: 256                           
       encoder_num_layers: 4                             
                input_dim: 40                            
         is_bidirectional: True                          
                 key_size: 128                           
                       lr: 0.001                         
               model_name: LAS_latest                    
                  n_epoch: 10                            
         save_latest_freq: 3                             
  

In [30]:
for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(dev_loader):
    utterances = utterances.permute(1, 0, 2)
    utterances = utterances.to(opt.device)
    labels = labels.to(opt.device)
    u_lens = u_lens.to(opt.device)
    l_lens = l_lens.to(opt.device)
    

In [11]:
torch.argmax(labels)

tensor(2516, device='cuda:0')

In [33]:
1106 - 256 * 4

82

In [37]:
word_list = list(word_list)

In [38]:
word_list.append('wtf???')

In [39]:
word_list = np.array(word_list)

In [40]:
word_list[labels[81].detach().cpu().numpy()]

array(['DOUBLE-QUOTE', "HAVEN'T", 'WE', 'ALREADY', 'GONE', 'wtf???', 'IN',
       'S.', 'B.', 'A.', 'BUDGET', 'CUTS', 'QUESTION-MARK',
       'DOUBLE-QUOTE', '<eos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>', '<sos>',
       '<sos>', '<sos>', '<sos>', '<sos>', '<sos>'], dtype='<U18')

In [41]:
np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')[-1]

array([b'DOUBLE-QUOTE', b"HAVEN'T", b'WE', b'ALREADY', b'GONE',
       b'OVERBOARD', b'IN', b'S.', b'B.', b'A.', b'BUDGET', b'CUTS',
       b'QUESTION-MARK', b'DOUBLE-QUOTE'], dtype='|S13')

In [46]:
word_dict['OVERBOARD']

17199

In [20]:
word_list[790]

'DOUBLE-QUOTE'

In [19]:
word_dict['PROPOSED']

818

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
options = BaseOptions()
opt = options.parser.parse_args(args = [])
# options.printer(opt)
speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")
encoder = Encoder(opt)
decoder = Decoder(opt)
encoder.to(opt.device)
decoder.to(opt.device)
# optimizer = Adam(list(encoder.parameters()) + list(decoder.parameters()), opt.lr)
# criterion = nn.CrossEntropyLoss(reduction = 'none')
# criterion.to(opt.device)

transcript_train = transform_letter_to_index(transcript_train)
transcript_valid = transform_letter_to_index(transcript_valid)

print("Transfer the transcript from letters to index sucessfully.....")
train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

Data Loading Sucessful.....
Transfer the transcript from letters to index sucessfully.....


In [4]:
# opt.encoder_num_layers = 3
# encoder = Encoder(opt)
# decoder = Decoder(opt)

In [4]:
opt.train_batch_size = 1

In [5]:
encoder.load_state_dict(torch.load('./LAS_latest/encoder_latest.pt'))
decoder.load_state_dict(torch.load('./LAS_latest/decoder_latest.pt'))

<All keys matched successfully>

In [6]:
train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)

In [7]:
encoder

Encoder(
  (blstm): ModuleList(
    (0): LSTM(40, 256, bias=False, bidirectional=True)
    (1): LSTM(512, 256, bias=False, bidirectional=True)
    (2): LSTM(512, 256, bias=False, bidirectional=True)
    (3): LSTM(512, 256, bias=False, bidirectional=True)
  )
  (pooling): ModuleList(
    (0): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (dropout): LockedDropout()
  (key_network): Linear(in_features=512, out_features=128, bias=True)
  (value_network): Linear(in_features=512, out_features=128, bias=True)
)

In [8]:
decoder

Decoder(
  (embedding): Embedding(34, 256)
  (attention): Attention(
    (linear): Linear(in_features=256, out_features=128, bias=True)
  )
  (lstm1): LSTMCell(384, 512)
  (lstm2): LSTMCell(512, 128)
  (character_prob): Linear(in_features=256, out_features=34, bias=True)
)

In [9]:
for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):
    break

In [11]:
utterances = utterances.permute(1, 0, 2)
utterances = utterances.to(opt.device)
labels = labels.to(opt.device)
u_lens = u_lens.to(opt.device)
l_lens = l_lens.to(opt.device)

In [12]:
encoder.cuda()
decoder.cuda()

Decoder(
  (embedding): Embedding(34, 256)
  (attention): Attention(
    (linear): Linear(in_features=256, out_features=128, bias=True)
  )
  (lstm1): LSTMCell(384, 512)
  (lstm2): LSTMCell(512, 128)
  (character_prob): Linear(in_features=256, out_features=34, bias=True)
)

In [13]:
hidden_states = [None, None]

In [14]:
keys, values, out_lens = encoder(utterances, u_lens)

In [15]:
keys = keys.permute(1, 0, 2)
values = values.permute(1, 0, 2)

In [16]:
prediction = torch.zeros(labels.size(0), 1).cuda()

In [17]:
char_embed = decoder.embedding(prediction.argmax()).cuda().unsqueeze(0)

In [ ]:
char_embed = decoder.embedding(prediction.argmax()).cuda().unsqueeze(0)

In [18]:
context, atten = decoder.attention(keys, char_embed, values, out_lens)

In [19]:
inp = torch.cat([char_embed, context], dim=1)
hidden_states[0] = decoder.lstm1(inp, hidden_states[0])

inp_2 = hidden_states[0][0]
hidden_states[1] = decoder.lstm2(inp_2,hidden_states[1])

output = hidden_states[1][0]
prediction = decoder.character_prob(torch.cat([output, context], dim=1))

In [20]:
prediction

tensor([[-4.9355,  1.8090,  1.5464,  0.5434,  1.3102,  0.3379,  0.6489, -0.6706,
          1.5793,  1.9630, -0.7702, -1.9915, -0.0747,  1.8338,  0.2829,  0.5961,
          0.4211, -1.3401, -0.3865,  1.3588,  3.0147, -0.6365, -1.1750,  1.2666,
         -2.2012, -1.0429, -3.1345, -2.9072, -3.1234, -2.3456, -4.7030, -3.9961,
         -1.6158, -2.6410]], device='cuda:0', grad_fn=<AddmmBackward>)

In [21]:
prediction.argmax()

tensor(20, device='cuda:0')

In [89]:
labels

tensor([[20,  8,  5, 32, 19,  5,  3, 21, 18,  9, 20,  9,  5, 19, 32,  1, 14,  4,
         32,  5, 24,  3,  8,  1, 14,  7,  5, 32,  3, 15, 13, 13,  9, 19, 19,  9,
         15, 14, 32, 19,  1,  9,  4, 32, 23,  5,  4, 14,  5, 19,  4,  1, 25, 32,
         20,  8,  1, 20, 32, 16,  5, 14, 14, 26, 15,  9, 12, 28, 19, 32, 13,  5,
         18,  7,  5, 18, 32, 16,  1,  3, 20, 32, 23,  9, 20,  8, 32,  7,  5, 20,
         20, 25, 32, 15,  9, 12, 32, 22,  9, 15, 12,  1, 20,  5,  4, 32, 19,  5,
          3, 21, 18,  9, 20,  9,  5, 19, 32, 12,  1, 23, 32,  3, 15, 13, 13,  1,
         32,  4,  5,  1, 12,  9, 14,  7, 32,  1, 32,  2, 12, 15, 23, 32, 20, 15,
         32,  9, 20, 19, 32, 12,  5,  7,  1, 12, 32,  3,  1, 19,  5, 32,  1,  7,
          1,  9, 14, 19, 20, 32, 20,  5, 24,  1,  3, 15, 32, 16,  5, 18,  9, 15,
          4, 32, 33]], device='cuda:0')

In [31]:
decoder.embedding(labels)[:, 1, :]

tensor([[ 1.4813e-01, -1.3363e-01,  6.2932e-02, -5.4767e-02,  3.2711e-02,
         -1.5171e-01, -5.6041e-02, -2.6125e-02,  3.1641e-02, -3.0940e-02,
         -5.4296e-02, -7.4529e-02, -5.1847e-02, -6.0010e-02,  4.0519e-02,
         -1.1589e-01, -3.9520e-02, -5.2895e-02, -4.9776e-02,  1.6528e-02,
         -1.0221e-01,  1.6935e-02,  7.7424e-02,  1.0448e-01,  1.5855e-02,
          2.3621e-02,  2.3680e-02, -1.0760e-02, -1.7462e-02, -9.5320e-02,
         -5.9259e-02,  1.1899e-03, -2.2156e-02, -1.2748e-01, -8.3798e-02,
          4.2732e-02,  1.0166e-01,  9.1426e-02, -6.3400e-02,  1.2062e-01,
         -4.0561e-02, -5.6035e-02, -1.2139e-02,  1.8392e-02,  1.1864e-02,
          5.4278e-02,  9.8208e-02,  6.6086e-02, -7.4572e-03,  7.5491e-03,
         -4.5364e-02, -7.8733e-02,  3.1196e-02,  6.2914e-02, -7.7238e-02,
          1.5153e-01, -3.7419e-02,  1.8120e-02, -1.3286e-01, -1.2729e-01,
         -1.3567e-02,  2.7741e-02, -1.4475e-01,  1.5018e-02, -1.0957e-01,
          5.2601e-02,  3.6138e-03, -8.

In [26]:
decoder.embedding(prediction.argmax(dim = 1))

tensor([[ 1.4813e-01, -1.3363e-01,  6.2932e-02, -5.4767e-02,  3.2711e-02,
         -1.5171e-01, -5.6041e-02, -2.6125e-02,  3.1641e-02, -3.0940e-02,
         -5.4296e-02, -7.4529e-02, -5.1847e-02, -6.0010e-02,  4.0519e-02,
         -1.1589e-01, -3.9520e-02, -5.2895e-02, -4.9776e-02,  1.6528e-02,
         -1.0221e-01,  1.6935e-02,  7.7424e-02,  1.0448e-01,  1.5855e-02,
          2.3621e-02,  2.3680e-02, -1.0760e-02, -1.7462e-02, -9.5320e-02,
         -5.9259e-02,  1.1899e-03, -2.2156e-02, -1.2748e-01, -8.3798e-02,
          4.2732e-02,  1.0166e-01,  9.1426e-02, -6.3400e-02,  1.2062e-01,
         -4.0561e-02, -5.6035e-02, -1.2139e-02,  1.8392e-02,  1.1864e-02,
          5.4278e-02,  9.8208e-02,  6.6086e-02, -7.4572e-03,  7.5491e-03,
         -4.5364e-02, -7.8733e-02,  3.1196e-02,  6.2914e-02, -7.7238e-02,
          1.5153e-01, -3.7419e-02,  1.8120e-02, -1.3286e-01, -1.2729e-01,
         -1.3567e-02,  2.7741e-02, -1.4475e-01,  1.5018e-02, -1.0957e-01,
          5.2601e-02,  3.6138e-03, -8.

In [1]:
import time
import torch
import numpy as np
from torch.optim import Adam
from options import *
from Encoder import *
from Decoder import *
from util import *
from Levenshtein import distance as levenshtein_distance

def training(opt, encoder, decoder, train_loader, val_loader):
    encoder.train()
    decoder.train()

    for epoch in range(opt.n_epoch):
        avg_loss = 0
        start = time.time()
        for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):
            utterances = utterances.permute(1, 0, 2)
            utterances = utterances.to(opt.device)
            labels = labels.to(opt.device)
            u_lens = u_lens.to(opt.device)
            l_lens = l_lens.to(opt.device)
            
            encoder.zero_grad()
            decoder.zero_grad()
            optimizer_encoder.zero_grad()
            optimizer_decoder.zero_grad()

            outputs, hidden = encoder(utterances, u_lens)
            hidden = (hidden[0].permute(1, 0, 2), hidden[1].permute(1, 0, 2))
            hidden = (hidden[0].reshape(hidden[0].size(0), -1), hidden[1].reshape(hidden[1].size(0), -1))

            predict_labels = decoder(hidden, labels)
            predict_labels = predict_labels.permute(0, 2, 1)
            loss = criterion(predict_labels, labels)
            mask = torch.arange(labels.size(1)).unsqueeze(0).to(opt.device) >= l_lens.unsqueeze(1)
            loss.masked_fill_(mask, 0.0)
            loss = loss.sum() / l_lens.sum()
            avg_loss += torch.exp(loss).item()
            loss.backward()

			# `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(encoder.parameters(), 0.25)   
            nn.utils.clip_grad_norm_(decoder.parameters(), 0.25)   

            if batch_idx % opt.display_freq == (opt.display_freq - 1):
                file_name = os.path.join('./' + opt.model_name, '{}.txt'.format(opt.model_name))
                with open(file_name, 'a') as opt_file:
                    opt_file.write('batch = {}, Perplexity = {}, Running time = {}'.format(batch_idx + 1, avg_loss / opt.display_freq, time.time() - start))
                    opt_file.write('\n')
                avg_loss = 0

            if batch_idx % (opt.display_freq * 10) == (opt.display_freq * 10 - 1):
                tmp_pred = transform_index_to_letter(predict_labels.unsqueeze(0))[0]
                tmp_true = np.array(letter_list)[labels[0].detach().cpu().numpy()]
                file_name_pred_train = os.path.join('./' + opt.model_name, '{}_pred_train.txt'.format(opt.model_name))
                with open(file_name_pred_train, 'a') as opt_file:
                    opt_file.write('prediction = {} Ground truth = {}'.format(tmp_pred, tmp_true))
                    opt_file.write('\n')
                
            optimizer_encoder.step()
            optimizer_decoder.step()

        validation(opt, encoder, decoder, val_loader)

def validation(opt, encoder, decoder, val_loader):
    encoder.eval()
    decoder.eval()

    start = time.time()
    running_loss = 0
    total_seq = 0
    score = 0
    for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(val_loader):
        utterances = utterances.permute(1, 0, 2)
        utterances = utterances.to(opt.device)
        labels = labels.to(opt.device)
        u_lens = u_lens.to(opt.device)
        l_lens = l_lens.to(opt.device)

        outputs, hidden = encoder(utterances, u_lens)
        hidden = (hidden[0].permute(1, 0, 2), hidden[1].permute(1, 0, 2))
        hidden = (hidden[0].reshape(hidden[0].size(0), -1), hidden[1].reshape(hidden[1].size(0), -1))

        predict_labels = decoder(hidden, labels)
        predict_labels = predict_labels.permute(0, 2, 1)
        loss = criterion(predict_labels, labels)
        mask = torch.arange(labels.size(1)).unsqueeze(0).to(opt.device) >= l_lens.unsqueeze(1)
        loss.masked_fill_(mask, 0.0)
        loss = loss.sum() / l_lens.sum()
        running_loss += torch.exp(loss).item()

        predict_labels = predict_labels.permute(0, 2, 1)
        for i in range(len(labels)):
            true_sentence = ''
            for j in range(l_lens[i] - 1):
                true_sentence += letter_list[labels[i][j]]

            predict_sentence = ''
            for j in range(len(predict_labels[i])):
                if predict_labels[i][j].argmax() == 33:
                    break
                predict_sentence += letter_list[predict_labels[i][j].argmax()]

            score += levenshtein_distance(true_sentence, predict_sentence)
            total_seq += 1

    print(score / total_seq)
    
    file_name = os.path.join('./' + opt.model_name, '{}.txt'.format(opt.model_name))
    with open(file_name, 'a') as opt_file:
        opt_file.write('='*16)
        opt_file.write('\n')
        opt_file.write('Perplexity = {}, Running time = {}'.format(running_loss / batch_idx, time.time() - start))
        opt_file.write('\n')
    
    predict_labels = predict_labels.permute(0, 2, 1)
    tmp_pred = transform_index_to_letter(predict_labels.unsqueeze(0))[0]
    tmp_true = np.array(letter_list)[labels[0].detach().cpu().numpy()]
    file_name_pred_val = os.path.join('./' + opt.model_name, '{}_pred_val.txt'.format(opt.model_name))
    with open(file_name_pred_val, 'a') as opt_file:
        opt_file.write('true = {}, predict = {}'.format(tmp_pred, tmp_true))
        opt_file.write('\n')

    encoder.train()
    decoder.train()

options = BaseOptions()
opt = options.parser.parse_args(args = [])
opt.model_name = 'base_model'
options.printer(opt)
speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")
encoder = SimpleEncoder(opt)
decoder = SimpleDecoder(opt)
encoder.to(opt.device)
decoder.to(opt.device)
print(decoder)
optimizer_encoder = Adam(encoder.parameters(), opt.lr, weight_decay = 1e-6)
optimizer_decoder = Adam(decoder.parameters(), opt.lr, weight_decay = 1e-6)
criterion = nn.CrossEntropyLoss(reduction = 'none')
criterion.to(opt.device)

transcript_train = transform_letter_to_index(transcript_train)
transcript_valid = transform_letter_to_index(transcript_valid)

print("Transfer the transcript from letters to index sucessfully.....")
train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

training(opt, encoder, decoder, train_loader, dev_loader)

----------------- Options ---------------
                 dataroot: ./data/                       
       decoder_hidden_dim: 512                           
       decoder_num_layers: 2                             
                   device: cuda                          
             display_freq: 10                            
                  dropout: 0.5                           
           embedding_size: 256                           
       encoder_hidden_dim: 256                           
       encoder_num_layers: 4                             
                input_dim: 40                            
         is_bidirectional: True                          
                 key_size: 128                           
                       lr: 0.001                         
               model_name: base_model                    	[default: LAS_latest]
                  n_epoch: 10                            
         save_latest_freq: 3                             
        

In [1]:
from torch.autograd import Variable

In [4]:
import torch

Variable(torch.ones((10))).requires_grad

False